In [1]:

%pprint
import sys
sys.path.insert(1, '../py')
from jobpostlib import (cu, datetime, duration, freq, hau, humanize, nu, scrfcu, slrcu, su, t0, time, wsu)
import os

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 48 seconds



----

In [2]:

nu.save_indeed_email_to_file(verbose=True)

['powershell.exe', '-ExecutionPolicy', 'Bypass', '-Command', 'Start-Process "C:\\Program Files\\Notepad++\\notepad++.exe" -ArgumentList "C:\\Users\\daveb\\OneDrive\\Documents\\GitHub\\job-hunting\\data\\html\\indeed_email.html"']


In [3]:

try: driver = wsu.get_driver(verbose=True); nu.beep(freq, int(duration/2))
finally:
    file_path = '../data/html/indeed_email.html'
    page_soup = wsu.get_page_soup(file_path)
    css_selector = 'table > tbody > tr > td > a > table > tbody > tr > td > a'
    link_soups_list = page_soup.select(css_selector)
    url_strs_set = set()
    for link_soup in link_soups_list: url_strs_set.add(link_soup['href'])
    display(len(url_strs_set))

Getting the FireFox driver
platform.system() = Windows
os.name = nt
gecko_driver_path = GeckoDriverManager().install() = C:\Users\daveb\.wdm\drivers\geckodriver\win64\v0.34.0\geckodriver.exe


4

In [4]:

files_list = []; close_notices = True
for url_str in url_strs_set:
    file_node_dict, files_list = su.load_indeed_posting_url(viewjob_url=url_str, driver=driver, files_list=files_list, close_notices=close_notices, verbose=False)
    close_notices = False
print(f'Fileing {len(files_list)} out of {len(url_strs_set)} postings complete. Delete the email.')

TimeoutException error closing cookie privacy notice: Message: 
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:193:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:511:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:136:16
Fileing 1 out of 4 postings complete. Delete the email.


In [5]:

t1 = time.time()
try: driver.close()
except Exception as e: print(f'{e.__class__.__name__} error: {str(e).strip()}')
cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='div', id='jobDescriptionText')
    for div_soup in row_div_list:
        child_strs_list = hau.get_navigable_children(div_soup, [])
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
duration_str = humanize.precisedelta(time.time() - t1, minimum_unit='seconds', format='%0.0f'); print(
    f'Populating {len(files_list)} out of {len(url_strs_set)} postings completed in {duration_str}'
); nu.beep(freq, duration)
files_list

Populating completed in 12 minutes and 24 seconds


['9006517badd8b5f0_Senior_Developer_Advocate_Burlingame_CA_Indeed_com.html']